**Name:** \_\_\_\_\_Huanchen Wang\_\_\_\_\_

**EID:** \_\_\_\_\_huancwang2\_\_\_\_\_

**Kaggle Team Name:** \_\_\_\_\_Unknown Wang\_\_\_\_\_

# CS5489 - Assignment 2 - Game Music Tagging
Due date: see Assignment 2 on Canvas

# FINAL SOLUTION

In [ ]:
%matplotlib inline
import matplotlib_inline   # setup output image format
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import csv
from scipy import io
import pickle
from IPython.display import Audio, display

In [ ]:
def showAudio(info):
    myfile = 'musicmp3/' + info['fname'] + '.mp3'
    if os.path.exists(myfile):
        display(Audio(myfile))
    else:
        print("*** mp3 file " + myfile + " could not be found ***")

def load_pickle(fname):
    f = open(fname, 'rb')
    out = pickle.load(f)
    f.close()
    return out

In [ ]:
train_tags  = load_pickle('musicdata/train_tags.pickle3')
train_mfccs = load_pickle('musicdata/train_mfccs.pickle3')
train_mels  = load_pickle('musicdata/train_mels.pickle3')
train_info  = load_pickle('musicdata/train_info.pickle3')

test_mfccs = load_pickle('musicdata/test_mfccs.pickle3')
test_mels  = load_pickle('musicdata/test_mels.pickle3')
test_info  = load_pickle('musicdata/test_info.pickle3')

In [ ]:
# compute delta MFCCs
def compute_delta_mfccs(mfccs):
    dmfccs = []
    for m in mfccs:
        tmp = m[1:] - m[0:-1]
        dm = hstack((m[0:-1], tmp))
        dmfccs.append(dm)
    return dmfccs

In [ ]:
train_dmfccs = compute_delta_mfccs(train_mfccs)
test_dmfccs  = compute_delta_mfccs(test_mfccs)

In [ ]:
tagnames, tagnames_counts = unique(concatenate(train_tags), return_counts=True)
for a,b in zip(tagnames, tagnames_counts):
    print("{}: {}".format(a, b))

In [ ]:
# convert list of tags into binary class labels
def tags2class(tags, tagnames):
    b = zeros(shape=(len(tags), len(tagnames)))
    for i,t in enumerate(tags):
        for j,n in enumerate(tagnames):
            if n in t:
                b[i,j] = 1
    return b

In [ ]:
# train_classes[i,j] = absence/presence of the j-th tag in the i-th sound
train_classes = tags2class(train_tags, tagnames)

In [ ]:
import csv

def write_csv_kaggle_tags(fname, tagnames, Yscores):
    # header
    tmp = [['Id']]
    for t in tagnames:
        tmp[0].append(t)    
    
    # add ID numbers for each Y, and usage if necessary
    for i in range(len(Yscores)):
        tmp2 = [(i+1)]
        for t in range(len(tagnames)):
            tmp2.append(Yscores[i,t])
        
        tmp.append(tmp2)
        
    # write CSV file
    f = open(fname, 'w')
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

## YOUR CODE and DOCUMENTATION HERE

In [ ]:
train_wav = load_pickle('train_wav.pkl')
test_wav = load_pickle('test_wav.pkl')
train_sr = load_pickle('train_sr.pkl')
test_sr = load_pickle('test_sr.pkl')

In [ ]:
import librosa

In [ ]:
def get_waveform_audio(music_data):
    wav_audio_list = []
    sample_rate_list = []
    for info in music_data:
        filename = 'musicmp3/' + info['fname'] + '.mp3'
        y, sr = librosa.load(filename, sr=16000)
        wav_audio_list.append(y)
        sample_rate_list.append(sr)
    return wav_audio_list, sample_rate_list

In [ ]:
train_wav, train_sr = get_waveform_audio(train_info)
test_wav, test_sr = get_waveform_audio(test_info)

In [ ]:

import datasets

from transformers import AutoFeatureExtractor,Wav2Vec2ForSequenceClassification


In [ ]:
train_X, val_X, train_Y, val_Y = model_selection.train_test_split(train_wav, train_classes, test_size=0.1, random_state=4487)

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/wav2vec2-base-960h')

train_dataset= datasets.Dataset.from_dict({'audio':train_X, 'label':train_Y})
val_dataset= datasets.Dataset.from_dict({'audio':val_X, 'label':val_Y})
test_dataset = datasets.Dataset.from_dict({'audio':test_wav})

In [ ]:
def wav_embedding(examples):
    return feature_extractor(examples["audio"], sampling_rate=16000, truncation=True, padding="max_length", max_length=160000, return_tensors="pt")

In [ ]:
train_dataset = train_dataset.map(wav_embedding, remove_columns=["audio"])
val_dataset = val_dataset.map(wav_embedding, remove_columns=["audio"])
test_dataset = test_dataset.map(wav_embedding, remove_columns=["audio"])

In [ ]:
X_train = [train_dataset[i]['input_values'] for i in range(len(train_dataset))]
Y_train = [train_dataset[i]['label'] for i in range(len(train_dataset))]
X_val = [val_dataset[i]['input_values'] for i in range(len(val_dataset))]
Y_val = [val_dataset[i]['label'] for i in range(len(val_dataset))]
X_test = [test_dataset[i]['input_values'] for i in range(len(test_dataset))]

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Conv1D, MaxPooling1D, Flatten
import keras_tuner as kt

In [ ]:
timesteps = 128
input_dim = 1250


In [ ]:
X_train = array([array(X_train[i]).reshape(timesteps, input_dim) for i in range(len(X_train))])
X_val = array([array(X_val[i]).reshape(timesteps, input_dim) for i in range(len(X_val))])
X_test = array([array(X_test[i]).reshape(timesteps, input_dim) for i in range(len(X_test))])
Y_train = array([array(Y_train[i]) for i in range(len(Y_train))])
Y_val = array([array(Y_val[i]) for i in range(len(Y_val))])

In [ ]:

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(19, activation='softmax'))


# Compile the model
opt = tf.optimizers.Adam(learning_rate=1e-4)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC')])

# Set up early stopping and history callbacks
early_stop = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
history = tf.keras.callbacks.History()

# Train the model
history = model.fit(X_train,Y_train, epochs=15, verbose = 1, batch_size=32,
                    validation_data=(X_val, Y_val),
                    callbacks=[early_stop, history])



In [ ]:
# tup = model.evaluate(X_val, Y_val)

y_pred = model.predict(expand_dims(X_test, axis=-1))

write_csv_kaggle_tags('submission.csv', tagnames, y_pred)